In [11]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Define multiple 2D functions for visualization
def f1(x, y):
    return x**2 + y**2

def f2(x, y):
    return np.sin(x) * np.cos(y)

def f3(x, y):
    return (1 - x)**2 + 100 * (y - x**2)**2  # Rosenbrock function

functions = {
    "Simple Quadratic": f1,
    "Trigonometric": f2,
    "Rosenbrock": f3
}

# Generate data
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(x, y)

# Define a simple neural network
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(2, 10)
        self.fc2 = nn.Linear(10, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

# Initialize model, loss function, and optimizer
model = SimpleNet()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Generate synthetic data
def generate_data(func):
    X_train = torch.randn(1000, 2) * 5
    y_train = torch.tensor(func(X_train[:, 0].numpy(), X_train[:, 1].numpy())).float().unsqueeze(1)
    return X_train, y_train

# Training function
def train(max_epochs, X_train, y_train):
    losses = []
    path = []
    for epoch in range(max_epochs):
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        
        # Get current position
        with torch.no_grad():
            w1, w2 = model.fc1.weight.data[0, :2]  # Get the first two weights of the first layer
            path.append((w1.item(), w2.item()))
    
    return losses, path

# Visualization function
def visualize(epochs, func, losses, path):
    Z = func(X, Y)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot loss curve
    ax1.plot(losses[:epochs])
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('Loss Curve')
    
    # Plot contour and optimization path
    contour = ax2.contour(X, Y, Z, levels=20)
    ax2.plot(*zip(*path[:epochs]), 'ro-')
    ax2.set_xlabel('w1')
    ax2.set_ylabel('w2')
    ax2.set_title('Optimization Path')
    
    plt.tight_layout()
    plt.show()

# Create and display widgets
max_epochs = 100
epoch_slider = widgets.IntSlider(min=1, max=max_epochs, step=1, value=10, description='Epochs:')
function_dropdown = widgets.Dropdown(options=list(functions.keys()), value="Simple Quadratic", description='Function:')
output = widgets.Output()

display(epoch_slider, function_dropdown, output)

# Global variables to store training results
current_function = "Simple Quadratic"
losses = []
path = []

# Update function for the widgets
def update(change):
    global current_function, losses, path
    
    with output:
        clear_output(wait=True)
        
        # Only retrain if the function has changed
        if change['owner'] == function_dropdown or not losses:
            current_function = function_dropdown.value
            X_train, y_train = generate_data(functions[current_function])
            losses, path = train(max_epochs, X_train, y_train)
        
        visualize(epoch_slider.value, functions[current_function], losses, path)

epoch_slider.observe(update, names='value')
function_dropdown.observe(update, names='value')

# Initial visualization
update({'owner': function_dropdown})


IntSlider(value=10, description='Epochs:', min=1)

Dropdown(description='Function:', options=('Simple Quadratic', 'Trigonometric', 'Rosenbrock'), value='Simple Q…

Output()